In [ ]:
import numpy as np
import pandas as pd
ALIGMENT = 50
def tHeader(num : int):
    print(f"TASK {num}".center(ALIGMENT,'-'))

In [ ]:
tHeader(1)

data = np.random.random(size=(10,5))
column_names = [chr(x) for x in range(ord('a'), ord('a')+data.shape[1])]
averages = [np.mean(x, where=[m > 0.3 for m in x]) for x in data]
data = np.column_stack((data, averages))
df = pd.DataFrame(data, columns=column_names + ['averages'])
df

In [ ]:
tHeader(2)
from dateutil.relativedelta import relativedelta

def months_diff(df):
    startProd, endProd = pd.to_datetime(df[['FirstProductionDate', 'CompletionDate']])
    df['FullMonthsFromStartToEnd'] = relativedelta(endProd, startProd).months
    return df

df = pd.read_csv('wells_info.csv')
df.insert(5, 'FullMonthsFromStartToEnd', '')
df.apply(months_diff, axis=1)


In [ ]:
tHeader(3)

from statistics import mode
df = pd.read_csv('wells_info_na.csv')

def remove_nans(column):
    if column.dtype == np.float64:
        swap_val = np.nanmedian(column)
    else:
        no_nan = list(filter(lambda x: x==x, column))
        swap_val = mode(no_nan)
    return column.fillna(swap_val)

df.apply(remove_nans)

In [ ]:
tHeader(4)

def sum_by12(df):
    total_liq = sum(df[:12]["Liquid"])
    total_gas = sum(df[:12]["Gas"])
    out = pd.Series(data=[total_liq,total_gas], index=['Liquid By 12 months', 'Gas By 12 months'], dtype=object)
    return out

wells_info = pd.read_csv("wells_info.csv")
prod_info  = pd.read_csv("production.csv")

total_prod_by_api = prod_info[["API", "Liquid", "Gas"]].groupby("API").sum()
m12_prod_by_api   = prod_info[["API", "Liquid", "Gas"]].groupby("API").apply(sum_by12)
pd.merge(wells_info, pd.merge(total_prod_by_api, m12_prod_by_api, on='API'), on='API')